In [1]:
# @title Helper functions for VLMap Creation

import os
import math

import open3d as o3d
import numpy as np
import cv2
from tqdm import tqdm
import torch
import torchvision.transforms as transforms
import clip

from scipy.spatial.transform import Rotation as R

from utils.clip_mapping_utils import load_semantic, load_obj2cls_dict, save_map, cvt_obj_id_2_cls_id, depth2pc_realsense, transform_pc, get_real_cam_mat, pos2grid_id, project_point, get_color
# from utils.clip_mapping_utils import load_pose

from lseg.modules.models.lseg_net import LSegEncNet
from lseg.additional_utils.models import resize_image, pad_image, crop_image


def load_depth(depth_filepath):
    with open(depth_filepath, 'rb') as f:
        depth = np.load(f)
    return depth / 1000
    # return depth

def load_pose(pose_filepath):
    with open(pose_filepath, 'rb') as f:
        full_pose = np.load(f)
        pos = np.array(full_pose[:3], dtype=float).reshape((3, 1))
        quat = full_pose[3:]
        # quat[[1,2]] = quat[[2,1]]
        r = R.from_quat(quat)
        rot = r.as_matrix()

        return pos, rot

# def get_color(rgb):
#     h,w = rgb.shape[:2]
#     y, x = np.meshgrid(np.arange(h), np.arange(w), indexing="ij")

#     x = x.reshape((-1))
#     y = y.reshape((-1))
#     colors = rgb[y,x]
#     # print(colors.shape)
#     return colors.T

def create_lseg_map_batch(img_save_dir, camera_translation, cs=0.05, gs=1000, depth_sample_rate=100):
    mask_version = 1 # 0, 1

    # crop_size = 480 # 480
    # base_size = 520 # 520
    # lang = "door,chair,ground,ceiling,window,other"
    # labels = lang.split(",")

    # loading models
    # device = "cuda:1" if torch.cuda.is_available() else "cpu"
    # print(device)
    # clip_version = "ViT-B/32"
    # clip_feat_dim = {'RN50': 1024, 'RN101': 512, 'RN50x4': 640, 'RN50x16': 768,
    #                 'RN50x64': 1024, 'ViT-B/32': 512, 'ViT-B/16': 512, 'ViT-L/14': 768}[clip_version]
    # print("Loading CLIP model...")
    # clip_model, preprocess = clip.load(clip_version, device = device)  # clip.available_models()
    # clip_model.to(device).eval()
    # # print("Clip model -", clip_model.is_cuda())
    # # x = torch.randn((5,5)).to(device=device)
    # # print("Tensor device - ", type(x))
    # # clip_model.eval()
    # lang_token = clip.tokenize(labels)
    # lang_token = lang_token.to(device)
    # with torch.no_grad():
    #     text_feats = clip_model.encode_text(lang_token)
    #     text_feats = text_feats / text_feats.norm(dim=-1, keepdim=True)
    # text_feats = text_feats.cpu().numpy()
    # model = LSegEncNet(lang, arch_option=0,
    #                     block_depth=0,
    #                     activation='lrelu',
    #                     crop_size=crop_size)
    # model_state_dict = model.state_dict()
    global data_root
    # checkpoint_path = os.path.join(data_root, "lseg/checkpoints/demo_e200.ckpt")

    # print("Loading lseg checkpoint")
    # pretrained_state_dict = torch.load(checkpoint_path, map_location = device)
    # print("Lseg loaded")
    # pretrained_state_dict = {k.lstrip('net.'): v for k, v in pretrained_state_dict['state_dict'].items()}
    # model_state_dict.update(pretrained_state_dict)
    # model.load_state_dict(pretrained_state_dict)

    # model.eval()
    # model = model.cuda()

    # norm_mean= [0.5, 0.5, 0.5]
    # norm_std = [0.5, 0.5, 0.5]
    # padding = [0.0] * 3
    # transform = transforms.Compose(
    #     [
    #         transforms.ToTensor(),
    #         transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    #     ]
    # )

    print(f"loading scene {img_save_dir}")
    rgb_dir = os.path.join(img_save_dir, "rgb")
    depth_dir = os.path.join(img_save_dir, "depth")
    pose_dir = os.path.join(img_save_dir, "pose")
    # semantic_dir = os.path.join(img_save_dir, "semantic")
    # obj2cls_path = os.path.join(img_save_dir, "obj2cls_dict.txt")

    rgb_list = sorted(os.listdir(rgb_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    depth_list = sorted(os.listdir(depth_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    pose_list = sorted(os.listdir(pose_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    # pose_list = sorted(os.listdir(pose_dir), key=lambda x: int(
    #     x.split("_")[-1].split(".")[0]))
    # semantic_list = sorted(os.listdir(semantic_dir), key=lambda x: int(
    #     x.split("_")[-1].split(".")[0]))

    rgb_list = [os.path.join(rgb_dir, x) for x in rgb_list]
    depth_list = [os.path.join(depth_dir, x) for x in depth_list]
    pose_list = [os.path.join(pose_dir, x) for x in pose_list]
    # semantic_list = [os.path.join(semantic_dir, x) for x in semantic_list]


    map_save_dir = os.path.join(img_save_dir, "map")
    os.makedirs(map_save_dir, exist_ok=True)
    color_top_down_save_path = os.path.join(map_save_dir, f"color_top_down_{mask_version}.npy")
    # gt_save_path = os.path.join(map_save_dir, f"grid_{mask_version}_gt.npy")
    grid_save_path = os.path.join(map_save_dir, f"grid_lseg_{mask_version}.npy")
    weight_save_path = os.path.join(map_save_dir, f"weight_lseg_{mask_version}.npy")
    obstacles_save_path = os.path.join(map_save_dir, "obstacles.npy")

    # obj2cls = load_obj2cls_dict(obj2cls_path)

    # initialize a grid with zero position at the center
    # color_top_down_height = (camera_height + 1) * np.ones((gs, gs), dtype=np.float32)
    # color_top_down = np.zeros((gs, gs, 3), dtype=np.uint8)
    # gt = np.zeros((gs, gs), dtype=np.int32)
    # grid = np.zeros((gs, gs, clip_feat_dim), dtype=np.float32)
    obstacles = np.ones((gs, gs), dtype=np.uint8)
    weight = np.zeros((gs, gs), dtype=float)

    # save_map(color_top_down_save_path, color_top_down)
    # save_map(gt_save_path, gt)
    # save_map(grid_save_path, grid)
    # save_map(weight_save_path, weight)
    # save_map(obstacles_save_path, obstacles)

    tf_list = []
    # data_iter = zip(rgb_list, depth_list, semantic_list, pose_list)
    data_iter = zip(rgb_list, depth_list, pose_list)
    pbar = tqdm(total=len(rgb_list))
    # load all images and depths and poses
    # print('here')
    pc_combined = []
    colors = []
    count = 0
    for data_sample in data_iter:
        count += 1
        # if count % 3 != 0:
        #     continue
        # rgb_path, depth_path, semantic_path, pose_path = data_sample
        # if count < 20:
        #     continue
        # if count > 22:
        #     break
        
        rgb_path, depth_path, pose_path = data_sample

        
        bgr = cv2.imread(rgb_path)
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

        # read pose
        # pos, rot = load_pose(pose_path)  # z backward, y upward, x to the right -> theirs
        pos, rot = load_pose(pose_path)  # x front, y left, z upward -> ours

        trans_cam = np.zeros((4,4))
        trans_cam[0, 2] = 1
        trans_cam[1, 0] = -1
        trans_cam[2, 1] = -1

        trans_cam[:3,3] = np.array(camera_translation)
        trans_cam[3,3] = 1
        # trans_cam

        pose = np.eye(4)
        pose[:3, :3] = rot
        pose[:3, 3] = pos.reshape(-1)

        # pose = pose @ trans_cam

        tf_list.append(pose)
        if len(tf_list) == 1:
            init_tf_inv = np.linalg.inv(tf_list[0])

        tf = init_tf_inv @ pose @ trans_cam

        # read depth
        depth = load_depth(depth_path)
        # print(f"depth shape = {depth.shape}")
        # break
        # read semantic
        # semantic = load_semantic(semantic_path)
        # semantic = cvt_obj_id_2_cls_id(semantic, obj2cls)

        # pix_feats = get_lseg_feat(model, rgb, labels, transform, crop_size, base_size, norm_mean, norm_std)
        
        # transform all points to the global frame
        pc, mask = depth2pc_realsense(depth)
        shuffle_mask = np.arange(pc.shape[1]) 
        np.random.shuffle(shuffle_mask)
        shuffle_mask = shuffle_mask[::depth_sample_rate]
        mask = mask[shuffle_mask]
        pc = pc[:, shuffle_mask]
        pc = pc[:, mask]
        pc_global = transform_pc(pc, tf)

        color_local = get_color(rgb)
        color_local = color_local[:, shuffle_mask]
        color_local = color_local[:, mask]

        if pc_combined == []:
            pc_combined = pc_global
            colors = color_local
        else:
            pc_combined = np.hstack((pc_combined, pc_global))
            # print(color_local.shape, colors.shape)
            colors = np.hstack((colors, color_local))


        # rgb_cam_mat = get_real_cam_mat(rgb.shape[0], rgb.shape[1])
        # # print(pix_feats.shape)
        # # feat_cam_mat = get_real_cam_mat(pix_feats.shape[2], pix_feats.shape[3])

        # # project all point cloud onto the ground
        # for i, (p, p_local) in enumerate(zip(pc_global.T, pc.T)):
        #     x, y = pos2grid_id(gs, cs, p[0], p[1])   # changed p[2] -> p[1], i.e, z -> y

        #     # ignore points projected to outside of the map and points that are 0.5 higher than the camera (could be from the ceiling)
        #     if x >= obstacles.shape[0] or y >= obstacles.shape[1] or \
        #         x < 0 or y < 0 or p_local[2] < -0.5:
        #         continue

        #     rgb_px, rgb_py, rgb_pz = project_point(rgb_cam_mat, p_local)
        #     rgb_v = rgb[rgb_py, rgb_px, :]
        #     # semantic_v = semantic[rgb_py, rgb_px]
        #     # if semantic_v == 40:
        #     #     semantic_v = -1
            
        #     # when the projected location is already assigned a color value before, overwrite if the current point has larger height
        #     if p_local[2] < color_top_down_height[y, x]:
        #         color_top_down[y, x] = rgb_v
        #         color_top_down_height[y, x] = p_local[2]
        #         # gt[y, x] = semantic_v

        #     # average the visual embeddings if multiple points are projected to the same grid cell
        #     # px, py, pz = project_point(feat_cam_mat, p_local)
        #     # if not (px < 0 or py < 0 or px >= pix_feats.shape[3] or py >= pix_feats.shape[2]):
        #     #     feat = pix_feats[0, :, py, px]
        #     #     grid[y, x] = (grid[y, x] * weight[y, x] + feat) / (weight[y, x] + 1)
        #     #     weight[y, x] += 1
            
        #     # build an obstacle map ignoring points on the floor (0 means occupied, 1 means free)
        #     if p_local[2] > camera_height:
        #         continue
        #     obstacles[y, x] = 0
        pbar.update(1)
        if count == 75:
            break
        # count += 1
        # if count == 500:
        #     break

    
    # save_map(color_top_down_save_path, color_top_down)
    # save_map(gt_save_path, gt)
    # save_map(grid_save_path, grid)
    # save_map(weight_save_path, weight)
    # save_map(obstacles_save_path, obstacles)
    # colors = colors.reshape((-1,3))
    np.save(os.path.join(map_save_dir, "pointcloud.npy"), pc_combined)
    np.save(os.path.join(map_save_dir, "colors.npy"), colors)
    return pc_combined, colors


# def get_lseg_feat(model: LSegEncNet, image: np.array, labels, transform, crop_size=480, \
#                  base_size=520, norm_mean=[0.5, 0.5, 0.5], norm_std=[0.5, 0.5, 0.5]):
#     vis_image = image.copy()
#     image = transform(image).unsqueeze(0).cuda()
#     img = image[0].permute(1,2,0)
#     img = img * 0.5 + 0.5
    
#     batch, _, h, w = image.size()
#     stride_rate = 2.0/3.0
#     stride = int(crop_size * stride_rate)

#     long_size = base_size
#     if h > w:
#         height = long_size
#         width = int(1.0 * w * long_size / h + 0.5)
#         short_size = width
#     else:
#         width = long_size
#         height = int(1.0 * h * long_size / w + 0.5)
#         short_size = height


#     cur_img = resize_image(image, height, width, **{'mode': 'bilinear', 'align_corners': True})

#     if long_size <= crop_size:
#         pad_img = pad_image(cur_img, norm_mean,
#                             norm_std, crop_size)
#         print(pad_img.shape)
#         with torch.no_grad():
#             outputs, logits = model(pad_img, labels)
#         outputs = crop_image(outputs, 0, height, 0, width)
#     else:
#         if short_size < crop_size:
#             # pad if needed
#             pad_img = pad_image(cur_img, norm_mean,
#                                 norm_std, crop_size)
#         else:
#             pad_img = cur_img
#         _,_,ph,pw = pad_img.shape #.size()
#         assert(ph >= height and pw >= width)
#         h_grids = int(math.ceil(1.0 * (ph-crop_size)/stride)) + 1
#         w_grids = int(math.ceil(1.0 * (pw-crop_size)/stride)) + 1
#         with torch.cuda.device_of(image):
#             with torch.no_grad():
#                 outputs = image.new().resize_(batch, model.out_c,ph,pw).zero_().cuda()
#                 logits_outputs = image.new().resize_(batch, len(labels),ph,pw).zero_().cuda()
#             count_norm = image.new().resize_(batch,1,ph,pw).zero_().cuda()
#         # grid evaluation
#         for idh in range(h_grids):
#             for idw in range(w_grids):
#                 h0 = idh * stride
#                 w0 = idw * stride
#                 h1 = min(h0 + crop_size, ph)
#                 w1 = min(w0 + crop_size, pw)
#                 crop_img = crop_image(pad_img, h0, h1, w0, w1)
#                 # pad if needed
#                 pad_crop_img = pad_image(crop_img, norm_mean,
#                                             norm_std, crop_size)
#                 with torch.no_grad():
#                     output, logits = model(pad_crop_img, labels)
#                 cropped = crop_image(output, 0, h1-h0, 0, w1-w0)
#                 cropped_logits = crop_image(logits, 0, h1-h0, 0, w1-w0)
#                 outputs[:,:,h0:h1,w0:w1] += cropped
#                 logits_outputs[:,:,h0:h1,w0:w1] += cropped_logits
#                 count_norm[:,:,h0:h1,w0:w1] += 1
#         assert((count_norm==0).sum()==0)
#         outputs = outputs / count_norm
#         logits_outputs = logits_outputs / count_norm
#         outputs = outputs[:,:,:height,:width]
#         logits_outputs = logits_outputs[:,:,:height,:width]
#     outputs = outputs.cpu()
#     outputs = outputs.numpy() # B, D, H, W
#     predicts = [torch.max(logit, 0)[1].cpu().numpy() for logit in logits_outputs]
#     pred = predicts[0]

#     return outputs

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
__file__:  /home2/laksh.nanwani/vlmaps-lab/examples/context.py
imported path: /home2/laksh.nanwani/vlmaps-lab


In [2]:
np.array([[1,2,3], [4,5,6]])[np.array([0,0]), np.array([1,0])]

array([2, 1])

In [3]:
root_dir = !pwd
print(root_dir)
root_dir = root_dir[0]
print(root_dir)
# !mkdir data
# %cd data
# !gdown 1wjuiVcO92Rqer5gLk-X7hINfe4PCMQmu
# !pip install tqdm
# !unzip -o 5LpN3gDmAk7_1.zip | tqdm --desc extracted --unit files --unit_scale --total `unzip -l 5LpN3gDmAk7_1.zip | tail -n 1 | xargs echo -n | cut -d' ' -f2` > /dev/null
import os
data_root = os.path.join(root_dir.split("home2")[0], "scratch/laksh.nanwani/vlmaps_data")

# checkpoint_dir = os.path.join(data_root, "checkpoints")
folder_name = "data_odom"
data_dir = os.path.join(data_root, folder_name)
# data_dir = os.path.join(data_root, "data_depth_issue")


# print(checkpoint_dir)
print(data_dir)

['/home2/laksh.nanwani/vlmaps-lab']
/home2/laksh.nanwani/vlmaps-lab
/scratch/laksh.nanwani/vlmaps_data/data_odom


In [4]:
# setup parameters
# @markdown meters per cell size
cs = 0.05 # @param {type: "number"}
# @markdown map resolution (gs x gs)
gs = 1000 # @param {type: "integer"}
# @markdown camera height (used for filtering out points on the floor)
# camera_height = 1.5 # @param {type: "number"}
# 0.51 - base_link -> camera_link
camera_translation = [0.2, -0.222, 0.512]
# @markdown depth pixels subsample rate
depth_sample_rate = 100 # @param {type: "integer"}
# @markdown data where rgb, depth, pose are loaded and map are saved
data_dir = data_dir # @param {type: "string"}

In [5]:
pcd, colors = create_lseg_map_batch(data_dir, camera_translation=camera_translation, cs=cs, gs=gs, depth_sample_rate=depth_sample_rate)

loading scene /scratch/laksh.nanwani/vlmaps_data/data_odom


  0%|          | 1/274 [00:00<02:08,  2.13it/s]/tmp/ipykernel_3328356/4168299532.py:227: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if pc_combined == []:
 27%|██▋       | 75/274 [00:29<01:17,  2.58it/s]


In [6]:
pcd.shape
# colors.shape

(3, 399606)

In [7]:
PC = o3d.geometry.PointCloud()
PC.points = o3d.utility.Vector3dVector(pcd.T)
o3d.visualization.draw_geometries([PC])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
